In [4]:
import pandas as pd
import ast
import os

In [12]:
import_cols = ['created_at', 'id', 'cat_fight']

user_infos = ['id', 'name', 'screen_name', 'location', 'description', 'url', 'followers_count', 'created_at', 'verified']

keep_cols = ['created_at', 'id', 'full_text', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted',
            'is_retweet', 'hashtags', 'user_mentions', 'urls'] + ["user_" + user_info for user_info in user_infos]

#savepath = os.path.join('C:/', 'data', 'poltweets', "tweets_flattened_20200127.gz")
#datapath = os.path.join('C:/', 'data', 'poltweets', "tweets_flattened_20200127.gz")
datapath = os.path.join('D:/', 'data', 'poltweets', "tweets_combined_20200127.gz")

In [66]:
df = pd.read_csv(datapath, nrows = 100, index_col = 0)

In [44]:
df.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status,withheld_in_countries,withheld_scope,withheld_copyright
0,Thu Oct 22 13:01:05 +0000 2020,1319262500743975000,1319262500743974914,RT @NHOERUP: Tak for at holde ved @TorstenGejl...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,0,False,False,da,NaN,NaN,NaN,NaN,NaN,NaN
1,Thu Oct 22 13:00:50 +0000 2020,1319262436105588700,1319262436105588737,RT @plejefamilierne: Det kan ikke lade sig gør...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,0,False,False,da,NaN,NaN,NaN,NaN,NaN,NaN
2,Thu Oct 22 13:00:32 +0000 2020,1319262363145687000,1319262363145687041,Det skriger til himlen.\nSocialministeren bad ...,False,"[0, 274]","{'hashtags': [{'text': 'dkpol', 'indices': [25...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,53,False,False,da,NaN,NaN,NaN,NaN,NaN,NaN
3,Thu Oct 22 12:32:30 +0000 2020,1319255308875739100,1319255308875739138,RT @alternativet_: Statsministeren sagde i nyt...,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,0,False,False,da,NaN,NaN,NaN,NaN,NaN,NaN
4,Thu Oct 22 11:40:38 +0000 2020,1319242255115391000,1319242255115390982,RT @alternativet_: Vi kan ikke være bekendt at...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,...,0,False,False,da,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
def fix_dicts(string):
    if not isinstance(string, dict):
        string_as_dict = ast.literal_eval(string)
        return(string_as_dict)
    else:
        return(string)
    
df['entities'] = df['entities'].apply(fix_dicts)
    
df = pd.concat([df, pd.json_normalize(df['entities'], max_level = 1)], axis = 1)

In [50]:
df.loc[2, 'hashtags']

[hashtag.get('text') for hashtag in df.loc[2, 'hashtags']]

['dkpol', 'dksocial']

In [67]:
def fix_dicts(string):
    if not isinstance(string, dict):
        try: 
            string_as_dict = ast.literal_eval(string)
        except ValueError:
            return(string)
        return(string_as_dict)
    else:
        return(string)
    
df = df.reset_index()
df['is_retweet'] = df['retweeted_status'].notna()
df['entities'] = df['entities'].apply(fix_dicts)
df['user'] = df['user'].apply(fix_dicts)

user_infos = list(df.loc[0, 'user'].keys())

for user_info in user_infos:
    colname = "user_" + user_info
    try:
        df[colname] = df['user'].apply(lambda x: x.get(user_info))
    except TypeError:
        df[colname] = np.nan
#    df[colname] = df['user'].apply(lambda x: x.get(user_info))

In [68]:
for name in ["user_" + user_info for user_info in user_infos]:
    print(df[name].head().isna())

0    False
1    False
2    False
3    False
4    False
Name: user_id, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_id_str, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_name, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_screen_name, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_location, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_description, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_url, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_entities, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_protected, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_followers_count, dtype: bool
0    False
1    False
2    False
3    False
4    False
Name: user_friends_count, dtype: bool
0    False
1    False
2    Fal